In [1]:
%pip install requests

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import json
import os

In [3]:
#Define path
SECRETS_FILE = '/Users/gmccurdy/repos/curriculum/scripts/utilities/secrets.json'

# Load secrets
with open(SECRETS_FILE, 'r') as file:
    secrets = json.load(file)

# Assign variables from the secrets
API_URL = secrets.get('API_URL')
API_KEY = secrets.get('API_KEY')
COURSE_ID = secrets.get('COURSE_ID_STATISTICS')

#Define headers
headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

In [4]:
# Create the quiz
quiz_title = input('Quiz Title')
quiz_description = input('Quiz Description')
quiz_data = {
    'quiz': {
        'title': quiz_title,
        'description': f'<p>{quiz_description}</p>',
        'quiz_type': 'assignment',
        'published': False
    }
}

create_quiz_url = f'{API_URL}/api/v1/courses/{COURSE_ID}/quizzes'
response = requests.post(create_quiz_url, headers=headers, json=quiz_data)

if response.status_code == 200:
    quiz = response.json()
    quiz_id = quiz['id']
    print(f'Quiz created successfully with ID: {quiz_id}')
else:
    print('Failed to create quiz')
    print(f"Status Code: {response.status_code}")
    print(response.text)

Quiz created successfully with ID: 44236


In [5]:
multiple_choice_questions = [
    {
        'question_name': 'Definition of Confidence Interval',
        'question_type': 'multiple_choice_question',
        'question_text': '''What is a confidence interval in statistics?''',
        'points_possible': 1,
        'answers': [
            {'text': 'A single value estimate of a population parameter.', 'weight': 0},
            {'text': 'A range of values derived from sample data that likely contains the true population parameter.', 'weight': 100},  # Correct answer
            {'text': 'The probability that a sample statistic is equal to the population parameter.', 'weight': 0},
            {'text': 'A graphical representation of sample data distribution.', 'weight': 0}
        ]
    },
    {
        'question_name': 'Components of Confidence Interval for Mean',
        'question_type': 'multiple_choice_question',
        'question_text': '''Which of the following components is NOT part of the confidence interval formula for a population mean when the population standard deviation is known?''',
        'points_possible': 1,
        'answers': [
            {'text': 'Sample mean ($\bar{x}$)', 'weight': 0},
            {'text': 'Z-score ($z^*$)', 'weight': 0},
            {'text': 'Population standard deviation ($\sigma$)', 'weight': 0},
            {'text': 'Sample median', 'weight': 100}  # Correct answer
        ]
    },
    {
        'question_name': 'Confidence Level Interpretation',
        'question_type': 'multiple_choice_question',
        'question_text': '''What does a 95% confidence level signify in the context of confidence intervals?''',
        'points_possible': 1,
        'answers': [
            {'text': 'There is a 95% probability that the population parameter lies within the interval.', 'weight': 0},
            {'text': '95% of the sample data falls within the confidence interval.', 'weight': 0},
            {'text': 'If we take 100 different samples, approximately 95 of the confidence intervals will contain the true population parameter.', 'weight': 100},  # Correct answer
            {'text': 'The margin of error is 5%.', 'weight': 0}
        ]
    },
    {
        'question_name': 'Standard Error for Proportion',
        'question_type': 'multiple_choice_question',
        'question_text': '''How is the standard error for a sample proportion ($\hat{p}$) calculated?''',
        'points_possible': 1,
        'answers': [
            {'text': '$SE = \frac{\sigma}{\sqrt{n}}$', 'weight': 0},
            {'text': '$SE = \frac{s}{\sqrt{n}}$', 'weight': 0},
            {'text': '$SE = \sqrt{ \frac{\hat{p}(1 - \hat{p})}{n} }$', 'weight': 100},  # Correct answer
            {'text': '$SE = z^* \times \sigma$', 'weight': 0}
        ]
    },
    {
        'question_name': 'Critical Value Selection',
        'question_type': 'multiple_choice_question',
        'question_text': '''Which critical value ($z^*$) corresponds to a 99% confidence level?''',
        'points_possible': 1,
        'answers': [
            {'text': '1.645', 'weight': 0},
            {'text': '1.960', 'weight': 0},
            {'text': '2.576', 'weight': 100},  # Correct answer
            {'text': '2.326', 'weight': 0}
        ]
    },
    {
        'question_name': 'Conditions for Using t-Distribution',
        'question_type': 'multiple_choice_question',
        'question_text': '''When should the t-distribution be used instead of the z-distribution for constructing a confidence interval for the mean?''',
        'points_possible': 1,
        'answers': [
            {'text': 'When the population standard deviation is known and the sample size is large.', 'weight': 0},
            {'text': 'When the population standard deviation is unknown and the sample size is small.', 'weight': 100},  # Correct answer
            {'text': 'When estimating a population proportion.', 'weight': 0},
            {'text': 'When the population distribution is normal regardless of sample size.', 'weight': 0}
        ]
    },
    {
        'question_name': 'Impact of Sample Size on Confidence Interval',
        'question_type': 'multiple_choice_question',
        'question_text': '''How does increasing the sample size from 30 to 100 affect the width of a 95% confidence interval for the population mean, assuming the population standard deviation remains constant?''',
        'points_possible': 1,
        'answers': [
            {'text': 'The confidence interval becomes wider.', 'weight': 0},
            {'text': 'The confidence interval becomes narrower.', 'weight': 100},  # Correct answer
            {'text': 'The confidence interval remains the same width.', 'weight': 0},
            {'text': 'The confidence level increases.', 'weight': 0}
        ]
    },
    {
        'question_name': 'Assumptions for Confidence Intervals',
        'question_type': 'multiple_choice_question',
        'question_text': '''Which of the following is NOT an assumption required for constructing valid confidence intervals for the mean?''',
        'points_possible': 1,
        'answers': [
            {'text': 'Random sampling of data.', 'weight': 0},
            {'text': 'Independence of observations.', 'weight': 0},
            {'text': 'Population distribution must be skewed.', 'weight': 100},  # Correct answer
            {'text': 'Known population standard deviation (for z-intervals).', 'weight': 0}
        ]
    },
    {
        'question_name': 'Margin of Error Definition',
        'question_type': 'multiple_choice_question',
        'question_text': '''What does the margin of error represent in a confidence interval?''',
        'points_possible': 1,
        'answers': [
            {'text': 'The exact difference between the sample statistic and the population parameter.', 'weight': 0},
            {'text': 'The maximum expected difference between the sample statistic and the population parameter.', 'weight': 100},  # Correct answer
            {'text': 'The probability that the sample statistic equals the population parameter.', 'weight': 0},
            {'text': 'The range of all possible sample statistics.', 'weight': 0}
        ]
    },
    {
        'question_name': 'Confidence Interval for Proportion Conditions',
        'question_type': 'multiple_choice_question',
        'question_text': '''Which conditions must be met to use the normal approximation for constructing a confidence interval for a population proportion?''',
        'points_possible': 1,
        'answers': [
            {'text': 'Sample size must be less than 30.', 'weight': 0},
            {'text': '$n\hat{p} \geq 10$ and $n(1 - \hat{p}) \geq 10$', 'weight': 100},  # Correct answer
            {'text': 'Population proportion must be exactly 0.5.', 'weight': 0},
            {'text': 'Sample standard deviation must be known.', 'weight': 0}
        ]
    }
]

In [6]:
essay_questions = [
    
]

In [7]:
numerical_questions = [
    
]

In [8]:
# Combine all questions, including the essay question
questions = multiple_choice_questions + numerical_questions + essay_questions

# Add questions to the quiz
for question in questions:
    create_question_url = f'{API_URL}/api/v1/courses/{COURSE_ID}/quizzes/{quiz_id}/questions'

    # Base question data
    question_data = {
        'question': {
            'question_name': question['question_name'],
            'question_type': question['question_type'],
            'question_text': question['question_text'],
            'points_possible': question.get('points_possible', 1)
            # Do not include 'answers' here
        }
    }

    # Include 'answers' only if they exist and the question is not an essay question
    if 'answers' in question and question['question_type'] != 'essay_question':
        question_data['question']['answers'] = question['answers']

    # Include 'general_feedback' if it exists (for essay questions)
    if 'general_feedback' in question:
        question_data['question']['general_feedback'] = question['general_feedback']

    # Send the request to create the question
    response = requests.post(create_question_url, headers=headers, json=question_data)

    if response.status_code == 201 or response.status_code == 200:
        print(f"Added question '{question['question_name']}' successfully.")
    else:
        print(f"Failed to add question '{question['question_name']}'")
        print(f"Status Code: {response.status_code}")
        print(response.text)

Added question 'Definition of Confidence Interval' successfully.
Added question 'Components of Confidence Interval for Mean' successfully.
Added question 'Confidence Level Interpretation' successfully.
Added question 'Standard Error for Proportion' successfully.
Added question 'Critical Value Selection' successfully.
Added question 'Conditions for Using t-Distribution' successfully.
Added question 'Impact of Sample Size on Confidence Interval' successfully.
Added question 'Assumptions for Confidence Intervals' successfully.
Added question 'Margin of Error Definition' successfully.
Added question 'Confidence Interval for Proportion Conditions' successfully.
